In [387]:
import sys

!{sys.executable} -m pip install joblib

# Imports

In [388]:
import pandas as pd # to interact with dataframes 
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, OneHotEncoder  #for preprocessing
import joblib #to save encoders and models 
import os #to interact with hardware and create folders 
import category_encoders as ce
import json
from collections import OrderedDict
import warnings
warnings.simplefilter(action='ignore')

# Constants

In [389]:
# Initialize filepaths 
CITYNAME_ENCODER_PATH = 'Objects/Encoders/LabelEncoder/cityname'
PETS_ENCODER_PATH = 'Objects/Encoders/LabelEncoder/pets/'
#PET_ENCODER_PATH = 'Objects/Encoders/OrdinalEncoder/pets/'
CATEGORY_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/category/'
FEE_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/fee/'
STORAGE_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/storage/'
PARKING_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/parking/'
GYM_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/gym/'
POOL_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/pool/'
WOODFLOORS_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/woodfloors/'
PATIO_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/patio/'
CLUBHOUSE_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/clubhouse/'
INTERNET_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/internet/'
GATED_ENCODER_PATH = 'Objects/Encoders/BinaryEncoder/gated/'
CLEAN_SQL_PATH = 'sql_files/clean/'
PREPROCESSED_FILE_PATH = 'preprocessed/'
COLUMNS_JSON = 'city_columns/'

# Reading the Data

In [390]:
 # read the cleaned data 
data = pd.read_csv(os.path.join(PREPROCESSED_FILE_PATH, '_All_States_Cleaned.csv'))

In [391]:
data.head(100) # see first 5 rows of the data 

,category,bathrooms,bedrooms,fee,pets_allowed,price,square_feet,cityname,state,with_storage,with_parking,with_gym,with_pool,with_woodfloors,with_patio,with_clubhouse,with_internet,with_gated
0,apartment,1.0,1.0,No,only cats,2195.0,542,Redondo Beach,CA,False,False,False,False,False,False,False,False,False
1,home,1.5,3.0,No,both cats and dogs,1250.0,1500,Newport News,VA,False,False,False,False,False,False,False,False,False
2,home,2.0,3.0,No,no pets,1395.0,1650,Raleigh,NC,False,False,False,False,False,False,False,False,False
3,apartment,1.0,2.0,No,both cats and dogs,1600.0,820,Vacaville,CA,False,False,False,False,False,False,False,False,False
4,apartment,1.0,1.0,No,both cats and dogs,975.0,624,Albuquerque,NM,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,apartment,1.0,2.0,No,both cats and dogs,1015.0,845,Omaha,NE,False,False,False,False,False,False,False,False,False
96,home,2.0,3.0,No,both cats and dogs,1495.0,1850,La Vista,NE,False,False,False,False,False,False,False,False,False
97,apartment,1.0,1.0,No,no pets,1103.0,652,Austin,TX,False,True,False,True,False,False,False,False,False
98,apartment,1.0,1.0,No,no pets,1032.0,600,Austin,TX,False,True,False,True,False,False,False,False,False


In [392]:
data.head()

,category,bathrooms,bedrooms,fee,pets_allowed,price,square_feet,cityname,state,with_storage,with_parking,with_gym,with_pool,with_woodfloors,with_patio,with_clubhouse,with_internet,with_gated
0,apartment,1.0,1.0,No,only cats,2195.0,542,Redondo Beach,CA,False,False,False,False,False,False,False,False,False
1,home,1.5,3.0,No,both cats and dogs,1250.0,1500,Newport News,VA,False,False,False,False,False,False,False,False,False
2,home,2.0,3.0,No,no pets,1395.0,1650,Raleigh,NC,False,False,False,False,False,False,False,False,False
3,apartment,1.0,2.0,No,both cats and dogs,1600.0,820,Vacaville,CA,False,False,False,False,False,False,False,False,False
4,apartment,1.0,1.0,No,both cats and dogs,975.0,624,Albuquerque,NM,False,False,False,False,False,False,False,False,False


In [393]:
def summarise_data(df):
    print("\nOverview")
    display(df.head())
    print("\nSummary")
    display(df.describe(include= "all"))
    print("\nNull Values")
    display(df.isnull().sum()/len(df))
    print("\nShape")
    display(df.shape)

In [394]:
summarise_data(data)


Overview


,category,bathrooms,bedrooms,fee,pets_allowed,price,square_feet,cityname,state,with_storage,with_parking,with_gym,with_pool,with_woodfloors,with_patio,with_clubhouse,with_internet,with_gated
0,apartment,1.0,1.0,No,only cats,2195.0,542,Redondo Beach,CA,False,False,False,False,False,False,False,False,False
1,home,1.5,3.0,No,both cats and dogs,1250.0,1500,Newport News,VA,False,False,False,False,False,False,False,False,False
2,home,2.0,3.0,No,no pets,1395.0,1650,Raleigh,NC,False,False,False,False,False,False,False,False,False
3,apartment,1.0,2.0,No,both cats and dogs,1600.0,820,Vacaville,CA,False,False,False,False,False,False,False,False,False
4,apartment,1.0,1.0,No,both cats and dogs,975.0,624,Albuquerque,NM,False,False,False,False,False,False,False,False,False



Summary


,category,bathrooms,bedrooms,fee,pets_allowed,price,square_feet,cityname,state,with_storage,with_parking,with_gym,with_pool,with_woodfloors,with_patio,with_clubhouse,with_internet,with_gated
count,99492,99492.000000,99492.000000,99492,99492,99492.000000,99492.000000,99492,99492,99492,99492,99492,99492,99492,99492,99492,99492,99492
unique,2,NaN,NaN,2,4,NaN,NaN,2980,52,2,2,2,2,2,2,2,2,2
top,apartment,NaN,NaN,No,both cats and dogs,NaN,NaN,Dallas,TX,False,False,False,False,False,False,False,False,False
freq,87087,NaN,NaN,99291,92452,NaN,NaN,2858,11257,77910,55680,62213,55996,90613,72990,80428,88441,90840
mean,NaN,1.444408,1.726058,NaN,NaN,1527.418003,956.430688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,0.548055,0.751212,NaN,NaN,911.094608,417.571522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,0.000000,0.000000,NaN,NaN,0.000000,101.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,1.000000,1.000000,NaN,NaN,1013.000000,729.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,1.000000,2.000000,NaN,NaN,1350.000000,900.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,2.000000,2.000000,NaN,NaN,1795.000000,1115.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Null Values


category           0.0
bathrooms          0.0
bedrooms           0.0
fee                0.0
pets_allowed       0.0
price              0.0
square_feet        0.0
cityname           0.0
state              0.0
with_storage       0.0
with_parking       0.0
with_gym           0.0
with_pool          0.0
with_woodfloors    0.0
with_patio         0.0
with_clubhouse     0.0
with_internet      0.0
with_gated         0.0
dtype: float64


Shape


(99492, 18)

In [395]:
data.value_counts('state')[:10]

state
TX    11257
CA    10311
VA     8284
NC     6300
CO     6290
FL     5775
MD     5280
MA     5030
OH     4905
GA     4752
dtype: int64

In [396]:
# create a copy of the subsets of the dataframes for each state.

In [397]:
tx = data[data['state'] == 'TX'].copy()
ca = data[data['state'] == 'CA'].copy()
va = data[data['state'] == 'VA'].copy()
nc = data[data['state'] == 'NC'].copy()
co = data[data['state'] == 'CO'].copy()
fl = data[data['state'] == 'FL'].copy()
ma = data[data['state'] == 'MA'].copy()
md = data[data['state'] == 'MD'].copy()

In [398]:
df_list_state = [tx, ca, va, nc, co, fl, ma, md] #store the dataframes in a list
states = ['TX', 'CA', 'VA', 'NC', 'CO', 'FL', 'MA', 'MD'] # store the state names in a list
df_dict_state = dict(zip(states, df_list_state)) #create a dictionary of dataframes with states as key

In [399]:
data.tail(100)

,category,bathrooms,bedrooms,fee,pets_allowed,price,square_feet,cityname,state,with_storage,with_parking,with_gym,with_pool,with_woodfloors,with_patio,with_clubhouse,with_internet,with_gated
99392,apartment,1.0,1.0,No,both cats and dogs,725.0,636,Greenville,SC,False,True,False,False,False,False,False,False,False
99393,apartment,1.5,2.0,No,both cats and dogs,1125.0,900,Greenville,SC,False,True,True,True,False,False,False,False,False
99394,apartment,1.0,2.0,No,both cats and dogs,1961.0,619,Miami,FL,False,False,True,True,False,True,False,False,False
99395,apartment,1.0,1.0,No,both cats and dogs,783.0,698,Greenville,SC,True,False,False,False,False,False,False,False,True
99396,apartment,1.0,1.0,No,both cats and dogs,2702.0,565,Sunnyvale,CA,True,True,True,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99487,apartment,1.0,1.0,No,both cats and dogs,780.0,605,Houston,TX,True,True,True,False,True,True,False,False,False
99488,apartment,2.0,2.0,No,both cats and dogs,813.0,921,Jacksonville,FL,True,True,True,True,False,True,True,False,False
99489,apartment,1.0,1.0,No,both cats and dogs,1325.0,650,San Diego,CA,False,False,False,False,False,False,False,False,False
99490,apartment,1.0,1.0,No,both cats and dogs,931.0,701,Huntersville,NC,False,False,True,True,False,False,False,False,False


In [400]:
data.pets_allowed.unique()

array(['only cats', 'both cats and dogs', 'no pets', 'only dogs'],
      dtype=object)

In [378]:
data.columns

Index(['category', 'bathrooms', 'bedrooms', 'fee', 'pets_allowed', 'price',
       'square_feet', 'cityname', 'state', 'with_storage', 'with_parking',
       'with_gym', 'with_pool', 'with_woodfloors', 'with_patio',
       'with_clubhouse', 'with_internet', 'with_gated'],
      dtype='object')

In [379]:
# for storing encoders while preprocessing 

fee_encoder_dict = {}
cityname_encoder_dict = {}
pets_encoder_dict = {}
storage_encoder_dict = {}
parking_encoder_dict = {}
gym_encoder_dict = {}
pool_encoder_dict = {}
woodfloors_encoder_dict = {}
patio_encoder_dict = {}
clubhouse_encoder_dict = {}
internet_encoder_dict = {}
gated_encoder_dict = {}
category_encoder_dict = {}

def preprocess(df_dict):
    '''
    Cleans and preprocesses the data for future use 
    Cleaning:
        - Removes duplicate rows (if they exist)
        - Removes the state column as the dataframes are already separated 
        - Removes outliers on the basis of price column using lower limit as (Q1-1.5*IQR) and 
        upper limit as (Q3+1.5*IQR)
        
    Args:
        df_dict - the dictionary containing states as keys and the dataframe corresponding to them as values 
    Returns:
        None 
        
    '''
    
    for state, df in df_dict.items():
            
        # Cleaning the data 
        # Dropping the state column
        df.drop(['state'], axis=1, inplace=True)
        # Dropping duplicate rows (if they exist)
        df.drop_duplicates(inplace=True)
        # Removing outliers : price
        desc = df['price'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['price']>=lower_lim)&(df['price']<=upper_lim)]
        # Removing outliers : square_feet
        desc = df['square_feet'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['square_feet']>=lower_lim)&(df['square_feet']<=upper_lim)]
        
       
        # Saving city name for each state : 
        
        city_col = df.cityname.to_list()
        city_col = list(map(lambda x : x.lower(), city_col))
        city_columns = OrderedDict.fromkeys(city_col)
        columns = {
                'data_columns' : list(city_columns)
            }
        
        filename = os.path.join(COLUMNS_JSON, f'{state}_cityname.json')
        if not os.path.exists(COLUMNS_JSON):
            os.makedirs(COLUMNS_JSON)
        with open(filename, "w+") as f:
            f.write(json.dumps(columns))
        
        
        
        
    
            
        # since the data is cleaned now, we can store it in clean_dict 
        clean_df = df.copy()
        
        # write the data to SQL file to insert into the CLEAN database
        clean_sql_filepath = os.path.join(CLEAN_SQL_PATH, f'{state}.sql')
        if not os.path.exists(CLEAN_SQL_PATH):
            os.makedirs(CLEAN_SQL_PATH)

        with open(clean_sql_filepath, 'w+') as f:    
            for idx, row in clean_df.iterrows():
                query = f"""
    INSERT INTO {state} ('category', 'bathrooms', 'bedrooms', 'fee', 'pets_allowed', 'price',
                         'square_feet', 'cityname', 'state', 'with_storage', 'with_parking',
                         'with_gym', 'with_pool', 'with_woodfloors', 'with_patio',
                         'with_clubhouse', 'with_internet', 'with_gated')
    VALUES 
    {tuple(row.values)};\n
                """
                f.write(query)
      
    
    
    
    
        
        # Preprocessing the data : pets_allowed
        '''
        pets_encoder = OrdinalEncoder(categories=[["no pets", "only cats", "only dogs", "both cats and dogs"]])
        df['pets_allowed'] = pets_encoder.fit_transform(df[['pets_allowed']]) 
        pet_encoder_dict[state] = pets_encoder
        filename = os.path.join(PET_ENCODER_PATH, f'{state}_pets_encoder.joblib')
        if not os.path.exists(PET_ENCODER_PATH):
            os.makedirs(PET_ENCODER_PATH) 
     
        if not os.path.exists(PET_ENCODER_PATH):
            joblib.dump(pets_encoder, 'os.path.join(PET_ENCODER_PATH, f'{state}_pets_encoder.joblib')')
            #joblib.dump(pets_encoder)
    
        
        # Preprocessing the data: cityname
      
       
        
        city_encoder = LabelEncoder()
        df['cityname'] = city_encoder.fit_transform(df['cityname'])
        city_encoder_dict[state] = city_encoder
                
       
        filename = os.path.join(CITY_ENCODER_PATH, f'{state}_city_encoder.pkl')
                
        if not os.path.exists(CITY_ENCODER_PATH):
            os.makedirs(CITY_ENCODER_PATH) 
            
        if not os.path.exists(CITY_ENCODER_PATH):
            joblib.dump(city_encoder)
        
        '''
        
        
        
        
        label_encoder_cols = ['pets_allowed' , 'cityname']
        label_enc_dict = {
            'pets_allowed' : pets_encoder_dict,
            'cityname' : cityname_encoder_dict
            }
        
        
        for col in label_encoder_cols:
            label_encoder = LabelEncoder()
            df[col] = label_encoder.fit_transform(df[col])
            label_enc_dict[col][state] = label_encoder
            
            
        paths = {
            'pets_allowed' : os.path.join(PETS_ENCODER_PATH, f'{state}_pets_encoder.pkl'),
            'cityname' : os.path.join(CITYNAME_ENCODER_PATH, f'{state}_cityname_encoder.pkl')    
                
        }
        
        if not os.path.exists(CITYNAME_ENCODER_PATH):
            os.makedirs(CITYNAME_ENCODER_PATH)
        
        if not os.path.exists(PETS_ENCODER_PATH):
            os.makedirs(PETS_ENCODER_PATH)
        
        
        
        for col in label_encoder_cols:
            joblib.dump(label_enc_dict[col][state], paths[col]) 
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        #filename = os.path.join(city_encoder_dict[state], f'{state}_city_encoder.joblib')
        #if not os.path.exists(CITY_ENCODER_PATH):
            #joblib.dump(city_encoder)
             #joblib.dump(city_encoder)
        

        
        #city_encoder_dict[state] = city_encoder
        #if not os.path.exists(CITY_ENCODER_PATH):
            #os.makedirs(CITY_ENCODER_PATH)
        
        
        # Preprocessing the data 
        binary_encoder_cols = ['fee','with_storage', 'with_parking','with_gym', 'with_pool', 'with_woodfloors'
                               , 'with_patio','with_clubhouse', 'with_internet', 'with_gated','category']  
        
        binary_enc_dict = {
            'fee' : fee_encoder_dict,
            'with_storage' : storage_encoder_dict,
            'with_parking' : parking_encoder_dict,
            'with_gym': gym_encoder_dict,
            'with_pool': pool_encoder_dict,
            'with_woodfloors': woodfloors_encoder_dict,
            'with_patio' : patio_encoder_dict,
            'with_clubhouse' : clubhouse_encoder_dict,
            'with_internet' : internet_encoder_dict,
            'with_gated' : gated_encoder_dict,
            'category' : category_encoder_dict
        }
        
        for col in binary_encoder_cols :
            binary_encoder = ce.BinaryEncoder(cols=[col]);

            # transform the data 
            df = binary_encoder.fit_transform(df)
            binary_enc_dict[col][state] = binary_encoder
            
            
            
        paths = {
            'fee' : os.path.join(FEE_ENCODER_PATH, f'{state}_fee_encoder.pkl'),
            'with_storage' : os.path.join(STORAGE_ENCODER_PATH, f'{state}_storage_encoder.pkl'),
            'with_parking' : os.path.join(PARKING_ENCODER_PATH, f'{state}_parking_encoder.pkl'),
            'with_gym': os.path.join(GYM_ENCODER_PATH, f'{state}_gym_encoder.pkl'),
            'with_pool': os.path.join(POOL_ENCODER_PATH, f'{state}_pool_encoder.pkl'),
            'with_woodfloors': os.path.join(WOODFLOORS_ENCODER_PATH, f'{state}_woodfloors_encoder.pkl'),
            'with_patio' : os.path.join(PATIO_ENCODER_PATH, f'{state}_patio_encoder.pkl'),
            'with_clubhouse' : os.path.join(CLUBHOUSE_ENCODER_PATH, f'{state}_clubhouse_encoder.pkl'),
            'with_internet' : os.path.join(INTERNET_ENCODER_PATH, f'{state}_internet_encoder.pkl'),
            'with_gated' : os.path.join(GATED_ENCODER_PATH, f'{state}_gated_encoder.pkl'),
            'category': os.path.join(CATEGORY_ENCODER_PATH, f'{state}_category_encoder.pkl')
        }
            
            
        
        if not os.path.exists(FEE_ENCODER_PATH): #check if the desired file path exists
            os.makedirs(FEE_ENCODER_PATH) #if not then make one
        
        if not os.path.exists(STORAGE_ENCODER_PATH):
            os.makedirs(STORAGE_ENCODER_PATH)
        
        if not os.path.exists(PARKING_ENCODER_PATH):
            os.makedirs(PARKING_ENCODER_PATH)
        
        if not os.path.exists(GYM_ENCODER_PATH):
            os.makedirs(GYM_ENCODER_PATH)
        
        if not os.path.exists(POOL_ENCODER_PATH):
            os.makedirs(POOL_ENCODER_PATH)
        
        
        if not os.path.exists(WOODFLOORS_ENCODER_PATH):
            os.makedirs(WOODFLOORS_ENCODER_PATH)
        
        if not os.path.exists(PATIO_ENCODER_PATH):
            os.makedirs(PATIO_ENCODER_PATH)
        
        
        if not os.path.exists(CLUBHOUSE_ENCODER_PATH):
            os.makedirs(CLUBHOUSE_ENCODER_PATH)
        
        
        if not os.path.exists(INTERNET_ENCODER_PATH):
            os.makedirs(INTERNET_ENCODER_PATH)
        
        
        if not os.path.exists(GATED_ENCODER_PATH):
            os.makedirs(GATED_ENCODER_PATH)
        
        if not os.path.exists(CATEGORY_ENCODER_PATH):
            os.makedirs(CATEGORY_ENCODER_PATH)
        
        
        for col in binary_encoder_cols:
            joblib.dump(binary_enc_dict[col][state], paths[col])
        
        
        
        
        # Saving the dataframes 
        if not os.path.exists(PREPROCESSED_FILE_PATH):
                os.makedirs(PREPROCESSED_FILE_PATH)
        df.to_csv(os.path.join(PREPROCESSED_FILE_PATH, f'{state}.csv'),index=False)   
        
      
        
        

In [380]:
preprocess(df_dict_state)

In [381]:
# Fetch the data  
data1 = pd.read_csv(os.path.join(PREPROCESSED_FILE_PATH, '_With_regions_States_Cleaned.csv')) # read the cleaned data 

In [382]:
tx_v = data1[data1['state'] == 'TX'].copy()
ca_v = data1[data1['state'] == 'CA'].copy()
va_v = data1[data1['state'] == 'VA'].copy()
nc_v = data1[data1['state'] == 'NC'].copy()
co_v = data1[data1['state'] == 'CO'].copy()
fl_v = data1[data1['state'] == 'FL'].copy()
ma_v = data1[data1['state'] == 'MA'].copy()
md_v = data1[data1['state'] == 'MD'].copy()

In [383]:
df_list_state1 = [tx_v, ca_v, va_v, nc_v, co_v, fl_v, ma_v, md_v] #store the dataframes in a list
states = ['TX', 'CA', 'VA', 'NC', 'CO', 'FL', 'MA', 'MD'] # store the state names in a list
df_dict_state1 = dict(zip(states, df_list_state1)) #create a dictionary of dataframes with states as key

In [384]:
def preprocess_viz(df_dict):
    '''
    Cleans and preprocesses the data for future use 
    Cleaning:
        - Removes duplicate rows (if they exist)
        - Removes the state column as the dataframes are already separated 
        - Removes outliers on the basis of price column using lower limit as (Q1-1.5*IQR) and 
        upper limit as (Q3+1.5*IQR)
        
    Args:
        df_dict - the dictionary containing states as keys and the dataframe corresponding to them as values 
    Returns:
        None 
        
    '''
    
    for state, df in df_dict.items():
            
        # Cleaning the data 
        # Dropping the city column
        df.drop(['state'], axis=1, inplace=True)
        # Dropping duplicate rows (if they exist)
        df.drop_duplicates(inplace=True)
        # Removing outliers : price
        desc = df['price'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['price']>=lower_lim)&(df['price']<=upper_lim)]
        # Removing outliers : square_feet
        desc = df['square_feet'].describe()
        q1 = desc.loc['25%']
        q3 = desc.loc['75%']
        iqr = q3-q1 
        lower_lim = q1-(1.5*iqr)
        upper_lim = q3+(1.5*iqr)
        df = df[(df['square_feet']>=lower_lim)&(df['square_feet']<=upper_lim)]
        
        # one hot encoding cityname
        #dummies = pd.get_dummies(df.cityname)
        #df = pd.concat([df,dummies], axis = 'columns')
        #df = df.drop('cityname',axis=1)
            
        # since the data is cleaned now, we can store it in clean_dict 
        clean_df = df.copy()
        # Saving the dataframes 
        if not os.path.exists(PREPROCESSED_FILE_PATH):
                os.makedirs(PREPROCESSED_FILE_PATH)
        df.to_csv(os.path.join(PREPROCESSED_FILE_PATH, f'{state}_viz.csv'),index=False)  

In [385]:
preprocess_viz(df_dict_state1)

In [386]:
        
        #cat_cols = ["cityname"]
        #cat_cols_encoded = []
        #for col in cat_cols:
            #cat_cols_encoded += [f"{col[0]}_{cat}" for cat in list(df[col].unique())]
        
        # Create an instance of One-hot-encoder 
        
        #city_encoder = OneHotEncoder(categories='auto',drop = 'first') 

                        
        #encoded_cols = city_encoder.fit_transform(df[cat_cols].fillna('Missing'))
        #df_enc = pd.DataFrame(encoded_cols)
        #df = pd.concat((df,df_enc),axis = 'columns')
        #df.drop(['cityname'], axis = 1, inplace = True) # Dropping the state column # Dropping the state column